# Fraud Detection Data

In [4]:
import warnings
from datetime import datetime
import os
import sys
import json
import pickle

import numpy as np
import pandas as pd
import plotly.express as px
import seaborn as sns
from geopy.distance import geodesic
from matplotlib import pyplot as plt
from summarytools import dfSummary

# Add the src directory to the Python path
sys.path.append(os.path.abspath(os.path.join('..', 'src')))

from utils.plots import FraudMap
from features.feature_engineering import generic_customer_spending_behaviour, general_customer_bahaviour, get_merchant_risk_rolling_window
from features.feature_transformation import encode, categorize_jobs

warnings.filterwarnings("ignore")

%load_ext autoreload
%autoreload 2


## Load Data

In [5]:
print("Load transaction data")
%time df = pd.read_csv("../data/raw/tr_fincrime_train.csv")
print("{0} transaction data loaded, containing {1} fraudulent transactions".format(len(df),df['is_fraud'].sum()))

Load transaction data
CPU times: user 6.69 s, sys: 1.09 s, total: 7.79 s
Wall time: 8.66 s
1296675 transaction data loaded, containing 7506 fraudulent transactions


In [6]:
df.head(5)

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,...,36.0788,-81.1781,3495,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0
1,1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,...,48.8878,-118.2105,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0
2,2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,...,42.1808,-112.2620,4154,Nature conservation officer,1962-01-19,a1a22d70485983eac12b5b88dad1cf95,1325376051,43.150704,-112.154481,0
3,3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,...,46.2306,-112.1138,1939,Patent attorney,1967-01-12,6b849c168bdad6f867558c3793159a81,1325376076,47.034331,-112.561071,0
4,4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,...,38.4207,-79.4629,99,Dance movement psychotherapist,1986-03-28,a41d7549acf90789359a9aa5346dcb46,1325376186,38.674999,-78.632459,0


In [7]:
# a general overview of the dataset
dfSummary(df)

No,Variable,Stats / Values,Freqs / (% of Valid),Graph,Missing
1,Unnamed: 0[int64],Mean (sd) : 648337.0 (374318.0)min < med < max:0.0 < 648337.0 < 1296674.0IQR (CV) : 648337.0 (1.7),"1,296,675 distinct values","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAHoAAAAyCAYAAACTUs/lAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjguMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8g+/7EAAAACXBIWXMAAAsTAAALEwEAmpwYAAAA30lEQVR4nO3UMQoCMRBAURXLqbOFjRfxEJ5gzpYTeArvsY2FqdOvrQgLaVbB/1+ZDGHgQ/bLsuz0/w6/XkDfcVy7KKVcI2IaeaT3fo6I2dnx2Y33eLbWbu9nq6EjYsrMx8jDtdZLZt6dHZ/deI/T55lfN4ShIQwNYWgIQ0MYGsLQEIaGMDSEoSEMDWFoCENDGBrC0BCGhjA0hKEhDA1haAhDQxgawtAQhoYwNIShIQwNYWgIQ0MYGsLQEIaGMDSEoSEMDWFoCENDGBrC0BCGhjA0hKEhDA1haAhDQxga4gWhg1ey46OS8wAAAABJRU5ErkJggg=="">",0(0.0%)
2,trans_date_trans_time[object],1. 2019-04-22 16:02:012. 2020-06-01 01:37:473. 2020-06-02 12:47:074. 2019-11-18 23:03:495. 2019-12-01 14:11:586. 2019-12-09 17:30:347. 2019-12-29 21:58:028. 2020-06-14 07:16:479. 2019-09-01 19:41:4510. 2019-07-08 18:30:5211. other,"4 (0.0%)4 (0.0%)4 (0.0%)3 (0.0%)3 (0.0%)3 (0.0%)3 (0.0%)3 (0.0%)3 (0.0%)3 (0.0%)1,296,642 (100.0%)","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAG8AAACzCAYAAACHOUI0AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjguMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8g+/7EAAAACXBIWXMAAAsTAAALEwEAmpwYAAACCklEQVR4nO3asW3EMBQFQVJwyFS4Ds71uAL17dx1yDWcEmKBmQoILPADPc37vgdNx+4H8Jx4YeKFiRcmXph4YeKFHXPO792P4JljjPHe/QiecTbDxAsTL0y8MPHCxAsTL0y8MPHCxAs7xhi/ux/BM9MPSF3OZphJKMwkFOZshokXJl6YeGHihYkXJl6YeGHihYkXZhIKMwmFOZthJqEwk1CYsxkmXph4YeKFiRcmXph4YeKFiRcmXphJKMwkFOZshpmEwkxCYc5mmHhh4oWJFyZemHhh4oWJFyZemHhhJqEwk1CYsxlmEgozCYU5m2HihYkXJl6YeGHihYkXJl6YeGHihZmEwkxCYc5mmEkozCQU5myGiRcmXph4YeKFiRcmXph4YeKFiRdmEgozCYU5m2EmoTCTUJizGSZemHhh4oWJFyZemHhh4oWJFyZemEkozCQU5myGmYTCTEJhzmaYeGHihYkXJl6YeGHihYkXJl6YeGHihdnzwux5Yc5mmD0vzJ4X5myGiRcmXph4YeKFiRcmXph4YeKFiRdmEgozCYU5m2EmoTCTUJizGSZemHhh4oWJFyZemHhh4oWJFyZemEkozCQU5myGzfM8f9Zar90P4XNfa63XdV1/ux/C55zNMPHCxAsTL0y8MPHCxAsTL0y8MPHCfNsMMwmFOZth4oWJFyZemHhh4oX9A5gQNK7t5DBkAAAAAElFTkSuQmCC"">",0(0.0%)
3,cc_num[int64],Mean (sd) : 417192042079726656.0 (1308806447000240384.0)min < med < max:60416207185.0 < 3521417320836166.0 < 4992346398065154048.0IQR (CV) : 4462212528794792.0 (0.3),983 distinct values,"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAHoAAAAyCAYAAACTUs/lAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjguMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8g+/7EAAAACXBIWXMAAAsTAAALEwEAmpwYAAAA7klEQVR4nO3csQ3CMBQAUUAUKVwnBUMwAUMwgWfzBHRsRIFr92EBLCWKwYi719pyvnRSyr+f53mn/3foPYC+41g7GMfxGkKYljxSSnnmnG/txlJr1dAhhCnG+FjySErp1G4kfYK/bghDQxgawtAQhoYwNIShIQwNYWgIQ0MYGsLQEIaGMDSEoSEMDWFoCENDGBrC0BCGhjA0hKEhDA1haAhDQxgawtAQhoYwNIShIQwNYWgIQ0MYGsLQEIaGMDSEoSEMDWFoCENDGBqiugtUv2XrEl5Dd7Qm3jAM5xjjfcndd0t4Dd3Ryg3Kly3fegGR2yDpZf4wjAAAAABJRU5ErkJggg=="">",0(0.0%)
4,merchant[object],"1. fraud_Kilback LLC2. fraud_Cormier LLC3. fraud_Schumm PLC4. fraud_Kuhn LLC5. fraud_Boyer PLC6. fraud_Dickinson Ltd7. fraud_Cummerata-Jones8. fraud_Kutch LLC9. fraud_Olson, Becker and Koch10. fraud_Stroman, Hudson and Erdm11. other","4,403 (0.3%)3,649 (0.3%)3,634 (0.3%)3,510 (0.3%)3,493 (0.3%)3,434 (0.3%)2,736 (0.2%)2,734 (0.2%)2,723 (0.2%)2,721 (0.2%)1,263,638 (97.5%)","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAG8AAACzCAYAAACHOUI0AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjguMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8g+/7EAAAACXBIWXMAAAsTAAALEwEAmpwYAAACIUlEQVR4nO3aoW3DUBhG0Wer0DQKKG/n6QSeLRN0n/KAGpu7qDwysa50zgRPutIP/Hk6jmPQNF/9AM4TL0y8MPHCxAsTL0y8sHmaps+rH8E58xjj4+pHcI6zGSZemHhh4oWJFyZemHhh4oWJFyZe2DzG+Ln6EZwz+QGpy9kMMwmFmYTCnM0w8cLECxMvTLww8cLECxMvTLww8cJMQmEmoTBnM8wkFGYSCnM2w8QLEy9MvDDxwsQLEy9MvDDxwsQLMwmFmYTCnM0wk1CYSSjM2QwTL0y8MPHCxAsTL0y8MPHCxAsTL8wkFGYSCnM2w0xCYSahMGczTLww8cLECxMvTLww8cLECxMvTLwwk1CYSSjM2QwzCYWZhMKczTDxwsQLEy9MvDDxwsQLEy9MvDDxwkxCYSahMGczzCQUZhIKczbDxAsTL0y8MPHCxAsTL0y8MPHCxAszCYWZhMKczTCTUJhJKMzZDBMvTLww8cLECxMvTLww8cLECxMvTLwwe16YPS/M2Qyz54XZ88KczTDxwsQLEy9MvDDxwsQLEy9MvDDxwkxCYSahMGczzCQU

In [8]:
# %%capture
# df.columns, df.describe(), df.info(), df.isna().sum(), df.duplicated().sum(), df.nunique()

## Univariate Analysis

### Daily Transactions

In [9]:
# Convert the 'trans_date_trans_time' column to datetime if not already done
df['trans_date_trans_time'] = pd.to_datetime(df['trans_date_trans_time'])
# Extract the date part from the 'trans_date_trans_time' column
df['trans_date'] = df['trans_date_trans_time'].dt.date

# Compute the number of transactions per day
transactions_per_day = df.groupby('trans_date').size().reset_index(name='num_transactions')

# Compute the number of fraudulent transactions per day
fraud_transactions_per_day = df[df['is_fraud'] == 1].groupby('trans_date').size().reset_index(name='num_fraud_transactions')

# Compute the number of fraudulent cards per day
fraud_cards_per_day = df[df['is_fraud'] == 1].groupby('trans_date')['cc_num'].nunique().reset_index(name='num_fraud_cards')

# Merge the dataframes to get a consolidated view
daily_fraud_stats = transactions_per_day.merge(fraud_transactions_per_day, on='trans_date', how='left').merge(fraud_cards_per_day, on='trans_date', how='left')

# Fill NaN values with 0
daily_fraud_stats.fillna(0, inplace=True)

In [10]:
import plotly.express as px
# Plot all three metrics on the same plot
fig = px.line(daily_fraud_stats, x='trans_date', y=['num_transactions', 'num_fraud_transactions', 'num_fraud_cards'],
              labels={'value': 'Count', 'variable': 'Metric'},
              title='Daily Transactions, Fraudulent Transactions, and Fraudulent Cards')
fig.show()

December has the peak transaction volume due to christimas and new year. Transaction volume show a homogenous behaviour on a granular monthly level. And typycally the peak of number of fraud transaction detected is at one week after the peak of transaction volum, meaning that there is (potentially) an average of one week delay. 

### Transaction Consistency 

In [26]:
# Calculate transaction counts
card_counts = df.groupby('cc_num')['amt'].count() 
card_amounts = df.groupby('cc_num')['amt'].sum()
merchant_counts = df.groupby('merchant')['amt'].count() 
merchant_amounts = df.groupby('merchant')['amt'].sum() 

card_amount_consistency = (card_amounts/card_counts).mean()/((card_amounts/card_counts).std()+0.1) #Add 0.1 to avoid explosion
merchant_amount_consistency = (merchant_amounts/merchant_counts).mean()/((merchant_amounts/merchant_counts).std()+0.1) #Add 0.1 to avoid explosion

print("\nCard Transaction Amount Consistency:\n", card_amount_consistency)
print("\nMerchantTransaction  Amount Consistency:\n", merchant_amount_consistency)


Card Transaction Amount Consistency:
 0.7566313765966384

MerchantTransaction  Amount Consistency:
 3.2026186971901316


### Transaction Density and Sparsity

In [27]:
 #Calculate sparsity as average transaction counts per card per day
cards_count_sparsity, per_card_count_sparsity = len(df['trans_date'].unique())/ card_counts, len(df['trans_date'].unique())/ card_counts.mean()
merchants_count_sparsity, per_merchant_count_sparsity = len(df['trans_date'].unique())/ merchant_counts, len(df['trans_date'].unique())/ merchant_counts.mean()

# #Calculate sparsity as average transaction amounts (per card) per day
cards_amount_sparsity, per_card_amount_sparsity = len(df['trans_date'].unique())/ card_amounts, len(df['trans_date'].unique())/ card_amounts.mean()
merchants_amount_sparsity, per_merchant_amount_sparsity = len(df['trans_date'].unique())/ merchant_amounts, len(df['trans_date'].unique())/ merchant_amounts.mean()

print("Card Transaction Count Sparsity:\n", per_card_count_sparsity)
print("\nMerchantTransaction Count Sparsity:\n", per_merchant_count_sparsity)
print("\nCard Transaction Amount Sparsity:\n", per_card_amount_sparsity)
print("\nMerchantTransaction  Amount Sparsity:\n", per_merchant_amount_sparsity)

Card Transaction Count Sparsity:
 0.40709584128636706

MerchantTransaction Count Sparsity:
 0.2869963560645497

Card Transaction Amount Sparsity:
 0.005786636097780993

MerchantTransaction  Amount Sparsity:
 0.004079490148283039


### Fraud by Location

In [17]:
fraud_transactions = df[df["is_fraud"] == 1]
fraud_map = FraudMap(fraud_transactions).run(port=8040)

### Fraud by Gender

In [29]:
# Calculate the fraud ratio by gender
fraud_ratio_by_gender = df.groupby("gender")["is_fraud"].mean().reset_index()
import plotly.express as px

# Plot the fraud ratio by gender using plotly
fig = px.bar(fraud_ratio_by_gender, x="gender", y="is_fraud", 
             title="Ratio of Fraud Transactions by Gender", 
             labels={"gender": "Gender", "is_fraud": "Fraud Ratio"},
             color="is_fraud", color_continuous_scale="viridis")

fig.update_layout(xaxis_tickangle=-45)
fig.show()

### Fraud by Job

I divide the jobs into 12 categories with the help from Gemini and Chatgpt: 
1. Healthcare & Medical
2. Engineering & Technology
3. Finance, Banking & Insurance
4. Education & Research
5. Creative Arts, Design & Media
6. Legal & Public Sector
7. Business, Management & Consultancy
8. Science & Research
9. Logistics, Transport & Supply Chain
10. Construction & Property
11. Hospitality, Tourism & Leisure
12. Other

In [30]:
# Load job normalisation gazetteer
with open('../jobs_by_category.json', 'r') as f:
    job_categories = json.load(f)

In [31]:
# normalize and categorize the job column 
df_cat_job = categorize_jobs(df_enc_gender, "job", job_categories)

In [32]:
fraud_by_job_category = df_cat_job.groupby("job_category")["is_fraud"].mean().reset_index(name="fraud_count")
# Sort the dataframe by fraud_count
fraud_by_job_category = fraud_by_job_category.sort_values(by="fraud_count", ascending=False)

# Plot the fraud transactions by job category using plotly
fig = px.bar(fraud_by_job_category, x="job_category", y="fraud_count", 
             title="Fraud Transactions by Job Category", 
             labels={"job_category": "Job Category", "fraud_count": "Number of Fraud Transactions"},
             color="fraud_count", color_continuous_scale="viridis")

fig.update_layout(xaxis_tickangle=-45)
fig.show()

### Fraud by Merchant Category

In [33]:
df_enc_gender.category.unique()

array(['misc_net', 'grocery_pos', 'entertainment', 'gas_transport',
       'misc_pos', 'grocery_net', 'shopping_net', 'shopping_pos',
       'food_dining', 'personal_care', 'health_fitness', 'travel',
       'kids_pets', 'home'], dtype=object)

In [34]:
# Calculate the fraud transaction rate by category
fraud_rate_by_category = df_cat_job.groupby("category")["is_fraud"].mean().reset_index()
fraud_rate_by_category = fraud_rate_by_category.sort_values(by="is_fraud", ascending=False)
# Plot the fraud transaction rate by category
fig = px.bar(fraud_rate_by_category, x='category', y='is_fraud', 
             title='Fraud Transaction Rate by Category', 
             labels={'category': 'Category', 'is_fraud': 'Fraud Transaction Rate'},
             color='is_fraud', color_continuous_scale='viridis')

fig.update_layout(xaxis_tickangle=-45)
fig.show()

## Multi-variate Analysis